In [784]:
import time
from math import floor
import numpy as np
import cv2
from scipy.sparse import csr_matrix
def preprocess_ncc_impl(image, ncc_size):
    """
    Prepare normalized patch vectors according to normalized cross
    correlation.

    This is a preprocessing step for the NCC pipeline.  It is expected that
    'preprocess_ncc' is called on every input image to preprocess the NCC
    vectors and then 'compute_ncc' is called to compute the dot product
    between these vectors in two images.

    NCC preprocessing has two steps.
    (1) Compute and subtract the mean.
    (2) Normalize the vector.

    The mean is per channel.  i.e. For an RGB image, over the ncc_size**2
    patch, compute the R, G, and B means separately.  The normalization
    is over all channels.  i.e. For an RGB image, after subtracting out the
    RGB mean, compute the norm over the entire (ncc_size**2 * channels)
    vector and divide.

    If the norm of the vector is < 1e-6, then set the entire vector for that
    patch to zero.

    Patches that extend past the boundary of the input image at all should be
    considered zero.  Their entire vector should be set to 0.

    Patches are to be flattened into vectors with the default numpy row
    major order.  For example, given the following
    2 (height) x 2 (width) x 2 (channels) patch, here is how the output
    vector should be arranged.

    channel1         channel2
    +------+------+  +------+------+ height
    | x111 | x121 |  | x112 | x122 |  |
    +------+------+  +------+------+  |
    | x211 | x221 |  | x212 | x222 |  |
    +------+------+  +------+------+  v
    width ------->

    v = [ x111, x121, x211, x221, x112, x122, x212, x222 ]

    see order argument in np.reshape

    Input:
        image -- height x width x channels image of type float32
        ncc_size -- integer width and height of NCC patch region; assumed to be odd
    Output:
        normalized -- heigth x width x (channels * ncc_size**2) array
    """
    # raise NotImplementedError()
    height, width, channel = image.shape
    normalized = np.zeros([height, width, channel * ncc_size ** 2])
    patch_size = int(ncc_size / 2)
    
    image_t = image.T
    for h in range(patch_size, height - patch_size):
        patches = []
        for w in range(patch_size, width - patch_size):
            patches.append(image_t[:, w - patch_size: w + patch_size + 1, h - patch_size : h + patch_size + 1])

        patches = np.array(patches)
        
        channel_matrix = np.reshape(np.transpose(patches, (0,1,3,2)), (len(patches), 3, ncc_size **2)).astype(np.float32)

        channel_matrix -= np.mean(channel_matrix, axis = 2, keepdims = True)
            
        for i in range(len(channel_matrix)):
            std = np.linalg.norm(channel_matrix[i])
            if std >= 1e-6:
                normalized[h, patch_size + i] = channel_matrix[i].reshape([-1]) / std

    return normalized


def compute_ncc_impl(image1, image2):
    """
    Compute normalized cross correlation between two images that already have
    normalized vectors computed for each pixel with preprocess_ncc.

    Input:
        image1 -- height x width x (channels * ncc_size**2) array
        image2 -- height x width x (channels * ncc_size**2) array
    Output:
        ncc -- height x width normalized cross correlation between image1 and
               image2.
    """
    ncc = np.einsum('ijk, ijk -> ij', image1, image2)
    return ncc

In [785]:
def compute_ncc(image1, image2):
    return compute_ncc_impl(image1, image2)


def preprocess_ncc(image, ncc_size):
    return preprocess_ncc_impl(image, ncc_size)

In [786]:
from imageio import imread
def ncc_full_shapes_test():
    ncc_size = 5

    image1 = imread('test_materials/ncc1.png')
    image2 = imread('test_materials/ncc2.png')

    n1 = preprocess_ncc(image1, ncc_size)
    n2 = preprocess_ncc(image2, ncc_size)

    result = compute_ncc(n1, n2)
    return n1, n2, result

In [787]:
def preprocess_ncc_impl(image, ncc_size):
    """
    Prepare normalized patch vectors according to normalized cross
    correlation.

    This is a preprocessing step for the NCC pipeline.  It is expected that
    'preprocess_ncc' is called on every input image to preprocess the NCC
    vectors and then 'compute_ncc' is called to compute the dot product
    between these vectors in two images.

    NCC preprocessing has two steps.
    (1) Compute and subtract the mean.
    (2) Normalize the vector.

    The mean is per channel.  i.e. For an RGB image, over the ncc_size**2
    patch, compute the R, G, and B means separately.  The normalization
    is over all channels.  i.e. For an RGB image, after subtracting out the
    RGB mean, compute the norm over the entire (ncc_size**2 * channels)
    vector and divide.

    If the norm of the vector is < 1e-6, then set the entire vector for that
    patch to zero.

    Patches that extend past the boundary of the input image at all should be
    considered zero.  Their entire vector should be set to 0.

    Patches are to be flattened into vectors with the default numpy row
    major order.  For example, given the following
    2 (height) x 2 (width) x 2 (channels) patch, here is how the output
    vector should be arranged.

    channel1         channel2
    +------+------+  +------+------+ height
    | x111 | x121 |  | x112 | x122 |  |
    +------+------+  +------+------+  |
    | x211 | x221 |  | x212 | x222 |  |
    +------+------+  +------+------+  v
    width ------->

    v = [ x111, x121, x211, x221, x112, x122, x212, x222 ]

    see order argument in np.reshape

    Input:
        image -- height x width x channels image of type float32
        ncc_size -- integer width and height of NCC patch region; assumed to be odd
    Output:
        normalized -- heigth x width x (channels * ncc_size**2) array
    """
    # raise NotImplementedError()
    height, width, channel = image.shape
    normalized = np.zeros([height, width, channel * ncc_size ** 2])
    patch_size = int(ncc_size / 2)
    
    image_t = image.T
    
    for h in range(patch_size, height - patch_size):
        for w in range(patch_size, width - patch_size):
            patch = image_t[:, w - patch_size : w + patch_size + 1, h - patch_size : h + patch_size + 1]

            channel_matrix = np.reshape(np.transpose(patch, (0,2,1)), (3, ncc_size **2)).astype(np.float32)

            channel_matrix -= np.mean(channel_matrix, axis = 1, keepdims = True)

            std = np.linalg.norm(channel_matrix)
            
            if std < 1e-6:
                continue
                
            normalized[h, w] = channel_matrix.reshape([-1]) / std
    print(normalized.shape)
            
    return normalized

In [788]:
%%time
n1, n2, result = ncc_full_shapes_test()

(256, 256, 75)
(256, 256, 75)
Wall time: 5.23 s


In [790]:
np.where(np.abs(result - correct) >= 1e-5)[0][:10]

array([], dtype=int64)

In [768]:
result[2]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [770]:
correct[2]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [774]:
correct = np.load('test_materials/ncc.npy')

assert result.shape == n1.shape[:2]
assert result.shape == n2.shape[:2]
assert (np.abs(result - correct) < 1e-5).all()

In [733]:
assert (np.abs(ans_n1 - n1) < 1e-5).all()
assert (np.abs(ans_n2 - n2) < 1e-5).all()

AssertionError: 

In [734]:
im = np.ones((5,5,3))
for i in range(len(im)):
    for j in range(len(im[0])):
        for c in range(3):
            im[i, j, c] = im[i, j, c] + i + 10 * j + 100 * c

In [735]:
im

array([[[  1., 101., 201.],
        [ 11., 111., 211.],
        [ 21., 121., 221.],
        [ 31., 131., 231.],
        [ 41., 141., 241.]],

       [[  2., 102., 202.],
        [ 12., 112., 212.],
        [ 22., 122., 222.],
        [ 32., 132., 232.],
        [ 42., 142., 242.]],

       [[  3., 103., 203.],
        [ 13., 113., 213.],
        [ 23., 123., 223.],
        [ 33., 133., 233.],
        [ 43., 143., 243.]],

       [[  4., 104., 204.],
        [ 14., 114., 214.],
        [ 24., 124., 224.],
        [ 34., 134., 234.],
        [ 44., 144., 244.]],

       [[  5., 105., 205.],
        [ 15., 115., 215.],
        [ 25., 125., 225.],
        [ 35., 135., 235.],
        [ 45., 145., 245.]]])

In [736]:
a = preprocess_ncc(im, 3)

In [737]:
banana

array([[  1.,  11.,  21.,  31.,  41.,   2.,  12.,  22.,  32.,  42.,   3.,
         13.,  23.,  33.,  43.,   4.,  14.,  24.,  34.,  44.,   5.,  15.,
         25.,  35.,  45.],
       [101., 111., 121., 131., 141., 102., 112., 122., 132., 142., 103.,
        113., 123., 133., 143., 104., 114., 124., 134., 144., 105., 115.,
        125., 135., 145.],
       [201., 211., 221., 231., 241., 202., 212., 222., 232., 242., 203.,
        213., 223., 233., 243., 204., 214., 224., 234., 244., 205., 215.,
        225., 235., 245.]], dtype=float32)

In [572]:
skip = 3
print(banana[0, 0:11:5])
print(banana[0, 1:12:5])
print(banana[0, 2:13:5])

[1. 2. 3.]
[11. 12. 13.]
[21. 22. 23.]


In [548]:
im[1:3, 0:2].T

array([[[  2.,   3.],
        [ 12.,  13.]],

       [[102., 103.],
        [112., 113.]],

       [[202., 203.],
        [212., 213.]]])

In [464]:
im.T[0:3, 0:2, 1:3]

array([[[  2.,   3.],
        [ 12.,  13.]],

       [[102., 103.],
        [112., 113.]],

       [[202., 203.],
        [212., 213.]]])

In [210]:
bb = np.array([np.array([np.array([1,2,3]),np.array([11,12,13]),np.array([21,22,23])]),
               np.array([np.array([101,102,103]),np.array([111,112,113]),np.array([121,122,123])]),
               np.array([np.array([201,202,203]),np.array([211,212,213]),np.array([221,222,223])])])
bb.shape

(3, 3, 3)

In [217]:
np.reshape(np.transpose(bb, (0,2,1)), (3, 9))

array([[  1,  11,  21,   2,  12,  22,   3,  13,  23],
       [101, 111, 121, 102, 112, 122, 103, 113, 123],
       [201, 211, 221, 202, 212, 222, 203, 213, 223]])

In [738]:
def correlated_ncc_test():
    ncc_size = 5
    ncc_half = int(ncc_size / 2)

    image1 = np.random.random((2 * ncc_size - 1, 2 * ncc_size - 1, 3))
    image2 = image1

    n1 = preprocess_ncc(image1, ncc_size)
    print(n1.shape)
    n2 = preprocess_ncc(image2, ncc_size)
    print(n2.shape)

    ncc = compute_ncc(n1, n2)
    print(ncc.shape)

    assert (np.abs(ncc[:ncc_half, :]) < 1e-5).all()
    assert (np.abs(ncc[-ncc_half:, :]) < 1e-5).all()
    assert (np.abs(ncc[:, :ncc_half]) < 1e-5).all()
    assert (np.abs(ncc[:, -ncc_half:]) < 1e-5).all()
    print(np.abs(ncc[ncc_half:-ncc_half, ncc_half:-ncc_half] - 1))
    assert (
        np.abs(ncc[ncc_half:-ncc_half, ncc_half:-ncc_half] - 1) < 1e-5).all()

In [739]:
correlated_ncc_test()

(9, 9, 75)
(9, 9, 75)
(9, 9)


AssertionError: 